In [7]:
# First, install Gurobi (Uncomment this line if Gurobi is not installed in your environment)
%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB
import random

# --------------------------------------------------------------------------
# 1. DEFINIÇÃO DOS DADOS BASE (Extraídos do Artigo - Situação I)
# --------------------------------------------------------------------------

num_strata = 5

base_revenue_c = [404.40, 286.5, 183.3, 102.57, 127.0]

area_limits_A = [21.66, 14.17, 11.33, 6.34, 7.83]

base_productivity_r = [50.57, 53.08, 43.09, 40.71, 38.69]
firewood_demand_min = 2300

base_labor_m = [87.5, 68.3, 58.0, 56.5, 48.6]
base_labor_M_max = 3500

# --------------------------------------------------------------------------
# 2. GERAÇÃO DE CENÁRIOS ESTOCÁSTICOS
# --------------------------------------------------------------------------

num_scenarios = 50  # Número de cenários a serem simulados
probabilities = [1.0 / num_scenarios] * num_scenarios

q_mo = 10.0  # Custo por hora de mão de obra extra (maior que o implícito na receita)
q_deficit = 50.0 # Penalidade por unidade de lenha não entregue

scenarios = []
print(f"Gerando {num_scenarios} cenários de incerteza...")

for i in range(num_scenarios):
    variability_factor = random.uniform(0.85, 1.15)

    scenario_c = [c * random.uniform(0.85, 1.15) for c in base_revenue_c]
    scenario_r = [r * random.uniform(0.85, 1.15) for r in base_productivity_r]
    scenario_M_max = base_labor_M_max * random.uniform(0.90, 1.10)

    scenarios.append({
        "c": scenario_c,
        "r_lenha": scenario_r,
        "M_max": scenario_M_max
    })
print("Cenários gerados com sucesso.\n")

# --------------------------------------------------------------------------
# 3. MODELAGEM COM GUROBI
# --------------------------------------------------------------------------

# Inicializa o modelo
model = gp.Model("GestaoFlorestalEstocastica")

# --- Variáveis de Decisão ---
# Variáveis de Primeiro Estágio (não dependem do cenário)
E = model.addVars(num_strata, vtype=GRB.CONTINUOUS, lb=0, name="E")

# Variáveis de Segundo Estágio (dependem do cenário)
y_mo = model.addVars(num_scenarios, vtype=GRB.CONTINUOUS, lb=0, name="y_mo")
y_deficit = model.addVars(num_scenarios, vtype=GRB.CONTINUOUS, lb=0, name="y_deficit")

# --- Função Objetivo ---
expected_revenue = gp.quicksum(
    probabilities[s] * gp.quicksum(scenarios[s]['c'][j] * E[j] for j in range(num_strata))
    for s in range(num_scenarios)
)

expected_recourse_cost = gp.quicksum(
    probabilities[s] * (q_mo * y_mo[s] + q_deficit * y_deficit[s])
    for s in range(num_scenarios)
)

model.setObjective(expected_revenue - expected_recourse_cost, GRB.MAXIMIZE)

# --- Restrições ---

# Restrições de Primeiro Estágio
for j in range(num_strata):
    model.addConstr(E[j] <= area_limits_A[j], f'Limite_Area_E{j+1}')

# Restrições de Segundo Estágio (para cada cenário)
for s in range(num_scenarios):
    labor_needed = gp.quicksum(base_labor_m[j] * E[j] for j in range(num_strata))
    model.addConstr(labor_needed <= scenarios[s]['M_max'] + y_mo[s], f'Restricao_MO_Cenario_{s+1}')

    firewood_produced = gp.quicksum(scenarios[s]['r_lenha'][j] * E[j] for j in range(num_strata))
    model.addConstr(firewood_produced >= firewood_demand_min - y_deficit[s], f'Restricao_Lenha_Cenario_{s+1}')

# --------------------------------------------------------------------------
# 4. RESOLUÇÃO E APRESENTAÇÃO DOS RESULTADOS
# --------------------------------------------------------------------------

# Resolve o problema
print("Resolvendo o modelo de otimização estocástica...")
model.optimize()
print("Modelo resolvido.\n")

# Exibe os resultados
if model.status == GRB.OPTIMAL:
    print(f"Status da Solução: Optimal (Código {model.status})\n")  # Correct usage to get status description

    print("--- Decisões Ótimas de Alocação de Área (1º Estágio) ---")
    print("Esta é a estratégia robusta a ser implementada, considerando as incertezas:")
    for j in range(num_strata):
        print(f"Área a ser explorada no Estrato E_{j+1}: {E[j].X:.4f} ha")

    print("\n--- Valor Esperado da Função Objetivo ---")
    print(f"Lucro Líquido Esperado: US$ {model.objVal:.2f}\n")

    print("--- Análise dos Custos de Recurso Esperados ---")
    total_expected_mo_cost = sum(probabilities[s] * q_mo * y_mo[s].X for s in range(num_scenarios))
    total_expected_deficit_cost = sum(probabilities[s] * q_deficit * y_deficit[s].X for s in range(num_scenarios))

    print(f"Custo esperado com contratação de mão de obra extra: US$ {total_expected_mo_cost:.2f}")
    print(f"Custo esperado com penalidades por déficit de lenha: US$ {total_expected_deficit_cost:.2f}")
else:
    # Print the status directly if not optimal
    print(f"O modelo não encontrou uma solução ótima. Status do modelo: {model.status}")


Gerando 50 cenários de incerteza...
Cenários gerados com sucesso.

Resolvendo o modelo de otimização estocástica...
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 105 rows, 105 columns and 605 nonzeros
Model fingerprint: 0x95f29221
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [2e-01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 4e+03]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 100 rows, 105 columns, 600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4640159e+04   1.742250e+03   0.000000e+00      0s
      46    1.2612942e+04   0.000000e+00   0.000000e+00      0s

Solved in 46 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.261294179e+04
Mode